In [1]:
#importing the datasets
movie_df <- read.csv("E:/IMDB-Dataset/movies.csv",stringsAsFactors=FALSE)
rating_df <- read.csv("E:/IMDB-Dataset/ratings.csv")

In [2]:
head(movie_df)

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller


In [3]:
nrow(movie_df)

[1] 10329

In [4]:
ncol(movie_df)

[1] 3

In [5]:
head(rating_df)

userId,movieId,rating,timestamp
1,16,4.0,1217897793
1,24,1.5,1217895807
1,32,4.0,1217896246
1,47,4.0,1217896556
1,50,4.0,1217896523
1,110,4.0,1217896150


In [6]:
nrow(rating_df)

[1] 105339

In [7]:
str(movie_df)

'data.frame':	10329 obs. of  3 variables:
 $ movieId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ title  : chr  "Toy Story (1995)" "Jumanji (1995)" "Grumpier Old Men (1995)" "Waiting to Exhale (1995)" ...
 $ genres : chr  "Adventure|Animation|Children|Comedy|Fantasy" "Adventure|Children|Fantasy" "Comedy|Romance" "Comedy|Drama|Romance" ...


In [8]:
str(rating_df)

'data.frame':	105339 obs. of  4 variables:
 $ userId   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ movieId  : int  16 24 32 47 50 110 150 161 165 204 ...
 $ rating   : num  4 1.5 4 4 4 4 3 4 3 0.5 ...
 $ timestamp: int  1217897793 1217895807 1217896246 1217896556 1217896523 1217896150 1217895940 1217897864 1217897135 1217895786 ...


To code both item-based and user-based collaborative recommendation system, we first need to create the User-Product matrix. We can do this easily with the pivot_wider function from tidyr.

In [10]:
library(dplyr)

We need a ratings matrix to build a recommender model with recommenderlab. This can, again, be easily done with the dcast() function in the reshape2 package.

In [11]:
library(reshape2)     

Warning message:
"package 'reshape2' was built under R version 3.6.3"

In [12]:
#Create ratings matrix. Rows = userId, Columns = movieId

In [13]:
ratingmat <- dcast(rating_df, userId~movieId, value.var = "rating", na.rm=FALSE)

In [14]:
ratingmat <- as.matrix(ratingmat[,-1]) #remove userIds

## Creation of the Recommender Model

The User-based Collaborative Filtering recommender model was created with recommenderlab with the below parameters and the ratings matrix:

Method: UBCF

Similarity Calculation Method: Cosine Similarity

Nearest Neighbors: 30

The predicted item ratings of the user will be derived from the 5 nearest neighbors in its neighborhood. When the predicted item ratings are obtained, the top 10 most highly predicted ratings will be returned as the recommendations.

In [16]:
#Normalize the data:
library(recommenderlab)

In [17]:
#Convert rating matrix into a recommenderlab sparse matrix
ratingmat <- as(ratingmat, "realRatingMatrix")

In [18]:
head(ratingmat)

1 x 10325 rating matrix of class 'realRatingMatrix' with 113 ratings.

In [19]:
#Normalize the data
ratingmat_norm <- normalize(ratingmat)

In [20]:
head(ratingmat_norm )

1 x 10325 rating matrix of class 'realRatingMatrix' with 113 ratings.
Normalized using center on rows.

In [22]:
#Create Recommender Model. "UBCF" stands for User-Based Collaborative Filtering

recommender_model <- Recommender(ratingmat_norm, method = "UBCF", param=list(method="Cosine",nn=30))


Warning message in .local(x, ...):
"x was already normalized by row!"

In [28]:
recom <- predict(recommender_model, ratingmat[1], n=10) 
#Obtain top 10 recommendations for 1st user in data

In [29]:
recom_list <- as(recom, "list") #convert recommenderlab object to readable list

In [30]:
head(recom_list)

[[1]]
 [1] "25"   "742"  "779"  "870"  "1000" "1060" "1356" "1539" "1542" "1544"

In [31]:
#Obtain recommendations
recom_result <- matrix(0,10)

for (i in c(1:10)){
 recom_result[i] <- movie_df[as.integer(recom_list[[1]][i]),2]
}

In [32]:
recom_result

Leaving Las Vegas (1995)
Gone with the Wind (1939)
Lady of Burlesque (1943)
"Murder, My Sweet (1944)"
"Terminator, The (1984)"
Forbidden Planet (1956)
Bent (1997)
"Nightmare on Elm Street 5: The Dream Child, A (1989)"
Friday the 13th Part 2 (1981)
Friday the 13th Part IV: The Final Chapter (1984)


And we have easily obtained the top 10 results for user 1! These were the movies recommended to user 1.